In [28]:
from PIL import Image
import numpy as np
import pickle
import os

def lzw_kompresija(sekvenca):
    rjecnik = dict() 
    
    for i in range(0,256):
        rjecnik[str(i)] = i  
        
    trenutni_kod = str(sekvenca[0])
    
    izlaz = [] 
    
    for piksel in sekvenca[1:]:
        temp_kod = trenutni_kod + ',' + str(piksel) 
        if temp_kod in rjecnik:
            trenutni_kod = temp_kod 
        else:
            izlaz.append(rjecnik[trenutni_kod]) 
            
            rjecnik[temp_kod] = len(rjecnik)  
            
            trenutni_kod = str(piksel)
            
          
    izlaz.append(rjecnik[trenutni_kod])
    return izlaz

def lzw_dekompresija(sekvenca):
    rjecnik = dict() 
    
    for i in range(0,256):
        rjecnik[i] = [i] 
    
    izlaz = []
    
    temp = sekvenca[0]
    
    izlaz.append(temp) 
    
    stari_kod = temp
    
    temp_sekvenca = []
    
    for novi_kod in sekvenca[1:]:
        if novi_kod in rjecnik:
            temp_sekvenca = rjecnik[novi_kod]
        else:
            temp_sekvenca = rjecnik[stari_kod] + [temp]
            
        izlaz.extend(temp_sekvenca)
    
        temp = temp_sekvenca[0]
    
        rjecnik[len(rjecnik)] = rjecnik[stari_kod] + [temp]
    
        stari_kod = novi_kod
    return izlaz

img = Image.open("lena.bmp")

img = img.convert("L")

vr_piksela = list(img.getdata())

sekvenca_piksela = vr_piksela[:]

kompresovana_sekvenca = lzw_kompresija(sekvenca_piksela)

kompresovana_sekvenca_np = np.array(kompresovana_sekvenca, dtype=np.uint16)

with open('kompresovana_slika.pkl', 'wb') as f:
    pickle.dump(kompresovana_sekvenca_np, f, protocol=4)

dekompresovana_sekvenca = lzw_dekompresija(kompresovana_sekvenca)

new_img = Image.new("L", img.size)

new_img.putdata(dekompresovana_sekvenca)

new_img.show()

velicina_originalne = os.path.getsize('lena.bmp')

velicina_kompresovane = os.path.getsize('kompresovana_slika.pkl')

CR=velicina_originalne/velicina_kompresovane

print(f"Odnos kompresije: {CR*100:.1f}%.")

SS=1-1/CR

print(f"Sačuvani prostor: {SS*100:.1f}%.")

suma=0

n=len(sekvenca_piksela)

for i in range(n):
    suma+=(sekvenca_piksela[i]-dekompresovana_sekvenca[i])**2

print(f"Srednja kvadratna greška: {suma/n}.")


Odnos kompresije: 375.2%.
Sačuvani prostor: 73.3%.
Srednja kvadratna greška: 0.0.
